In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report

import random
import numpy as np
import torch

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  
torch.backends.cudnn.deterministic = True  
torch.backends.cudnn.benchmark = False 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMAGE_DIR = 'D:\\PAD-UFES\\images'  
METADATA_PATH = 'D:\\PAD-UFES\\metadata.csv'

metadata = pd.read_csv(METADATA_PATH)

def preprocess_metadata(metadata):
    metadata = metadata.fillna('UNK')

    boolean_cols = [
        'smoke',
        'drink',
        'pesticide',
        'skin_cancer_history',
        'cancer_history',
        'has_piped_water',
        'has_sewage_system',
        'itch',
        'grew',
        'hurt',
        'changed',
        'bleed',
        'elevation',
        'biopsed',
    ]
    # Ensure columns are strings and lowercase
    for col in boolean_cols:
        metadata[col] = metadata[col].astype(str).str.lower()
    
    # Map boolean columns to 1/0/-1
    boolean_mapping = {'true': 1, 'false': 0, 'unk': -1}
    for col in boolean_cols:
        metadata[col] = metadata[col].map(boolean_mapping)
    
    # Handle categorical variables
    categorical_cols = [
        'background_father',
        'background_mother',
        'gender',
        'region',
        'diagnostic',
    ]
    # Convert categorical columns to string
    for col in categorical_cols:
        metadata[col] = metadata[col].astype(str)
    
    # One-hot encode categorical variables
    metadata_encoded = pd.get_dummies(metadata[categorical_cols])
    
    # Normalize numerical variables
    numerical_cols = ['age', 'fitspatrick', 'diameter_1', 'diameter_2']
    # Ensure numerical columns are numeric
    for col in numerical_cols:
        metadata[col] = pd.to_numeric(metadata[col], errors='coerce')
    # Fill NaNs in numerical columns with the mean
    metadata[numerical_cols] = metadata[numerical_cols].fillna(metadata[numerical_cols].mean())
    # Scale numerical columns
    scaler = StandardScaler()
    metadata_numeric = metadata[numerical_cols]
    metadata_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(metadata_numeric), columns=numerical_cols
    )
    
    # Combine all metadata features
    metadata_processed = pd.concat(
        [metadata_numeric_scaled.reset_index(drop=True),
         metadata_encoded.reset_index(drop=True),
         metadata[boolean_cols].reset_index(drop=True)], axis=1
    )
    
    return metadata_processed

# Preprocess metadata
metadata_processed = preprocess_metadata(metadata)

def get_image_paths(metadata, image_dir):
    image_paths = []
    for idx, row in metadata.iterrows():
        filename = row['img_id']
        # Ensure filename is a string
        filename = str(filename)
        # Check if filename has an extension
        if not filename.endswith(('.jpg', '.jpeg', '.png')):
            # Try common extensions
            possible_extensions = ['.jpg', '.jpeg', '.png']
            found = False
            for ext in possible_extensions:
                filepath = os.path.join(image_dir, filename + ext)
                if os.path.isfile(filepath):
                    image_paths.append(filepath)
                    found = True
                    break
            if not found:
                print(f"Image file not found for ID: {filename}")
                image_paths.append(None)
        else:
            filepath = os.path.join(image_dir, filename)
            if os.path.isfile(filepath):
                image_paths.append(filepath)
            else:
                print(f"Image file not found: {filepath}")
                image_paths.append(None)
    metadata['ImagePath'] = image_paths
    return metadata

metadata = get_image_paths(metadata, IMAGE_DIR)

# Remove entries with missing images
metadata = metadata[metadata['ImagePath'].notnull()]
metadata_processed = metadata_processed.loc[metadata.index].reset_index(drop=True)
metadata = metadata.reset_index(drop=True)

# Drop diagnostic-related columns from features
diagnostic_cols = ['diagnostic_ACK', 'diagnostic_BCC', 'diagnostic_MEL', 'diagnostic_NEV', 'diagnostic_SCC', 'diagnostic_SEK']
metadata_processed = metadata_processed.drop(columns=diagnostic_cols)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(metadata['diagnostic'])
num_classes = len(label_encoder.classes_)

# Split data into features and labels
X_meta = metadata_processed.reset_index(drop=True)
X_img_paths = metadata['ImagePath'].reset_index(drop=True)
y = pd.Series(y_encoded)

X_train_meta, X_temp_meta, X_train_img_paths, X_temp_img_paths, y_train, y_temp = train_test_split(
    X_meta,
    X_img_paths,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_val_meta, X_test_meta, X_val_img_paths, X_test_img_paths, y_val, y_test = train_test_split(
    X_temp_meta,
    X_temp_img_paths,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

# Load augmented metadata + image paths
aug_meta_df   = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
aug_labels_df = pd.read_csv("D:/PAD-UFES/augmented_labels.csv")

# Combine augmented samples with training set
X_train_meta_final = pd.concat([X_train_meta, aug_meta_df], ignore_index=True)
X_train_img_paths_final = pd.concat([X_train_img_paths.reset_index(drop=True),
                                     aug_labels_df['ImagePath']], ignore_index=True)
y_train_final = pd.concat([y_train.reset_index(drop=True),
                           aug_labels_df['Label']], ignore_index=True)

class PADUFESDataset(Dataset):
    def __init__(self, img_paths, meta_data, labels, transform=None):
        self.img_paths = img_paths.reset_index(drop=True)
        self.meta_data = meta_data.reset_index(drop=True)
        self.labels = pd.Series(labels).reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        meta = torch.tensor(self.meta_data.iloc[idx].values.astype(np.float32))
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return image, meta, label

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    transforms.RandomRotation(70),          
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = PADUFESDataset(X_train_img_paths_final, X_train_meta_final, y_train_final, transform=train_transform)
val_dataset = PADUFESDataset(X_val_img_paths, X_val_meta, y_val, transform=val_test_transform)
test_dataset = PADUFESDataset(X_test_img_paths, X_test_meta, y_test, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"\n✅ Loaded Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class EarlyFusionMobileNetV3(nn.Module):
    def __init__(self, input_dim_meta, num_classes, model_size='large'):
        super().__init__()
        
        # Embed metadata to smaller spatial dimensions
        self.meta_embed = nn.Sequential(
            nn.Linear(input_dim_meta, 56 * 56),
            nn.ReLU(),
            nn.BatchNorm1d(56 * 56),
            nn.Dropout(0.3)
        )
        
        # Load MobileNetV3
        model_name = f'mobilenetv3_{model_size}'  # 'mobilenetv3_large' or 'mobilenetv3_small'
        self.mobilenet = timm.create_model("mobilenetv3_large_100", pretrained=True, num_classes=6)
        
        # Store the original first conv layer
        first_conv = self.mobilenet.conv_stem
        
        # Modify first conv layer to accept 4 channels (RGB + metadata)
        self.mobilenet.conv_stem = nn.Conv2d(
            4, 
            first_conv.out_channels,
            kernel_size=first_conv.kernel_size,
            stride=first_conv.stride,
            padding=first_conv.padding,
            bias=False
        )
        
        # Initialize new conv layer
        with torch.no_grad():
            # Copy weights for RGB channels
            self.mobilenet.conv_stem.weight[:, :3] = first_conv.weight
            # Initialize metadata channel with mean of RGB weights * small factor
            self.mobilenet.conv_stem.weight[:, 3:] = first_conv.weight.mean(dim=1, keepdim=True) * 0.1
        
        # Modify classifier
        in_features = self.mobilenet.classifier.in_features
        self.mobilenet.classifier = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(in_features, num_classes)
        )
        
    def forward(self, img, meta):
        batch_size = img.shape[0]
        
        # Process metadata
        meta_features = self.meta_embed(meta)
        meta_reshaped = meta_features.view(batch_size, 1, 56, 56)
        
        meta_upsampled = F.interpolate(
            meta_reshaped,
            size=(224, 224),
            mode='bilinear',
            align_corners=False
        )
        
        # Concatenate along channel dimension
        combined_input = torch.cat([img, meta_upsampled], dim=1)
        
        # Forward pass through MobileNetV3
        output = self.mobilenet(combined_input)
        
        return output

input_dim_meta = X_train_meta.shape[1]  
num_classes = 6  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EarlyFusionMobileNetV3(input_dim_meta, num_classes).to(device)
print(device)
from torchinfo import summary
summary(model=model, 
        input_size=[(16, 3, 224, 224), (16, input_dim_meta)],  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
print(input_dim_meta)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def test(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            outputs = model(imgs, metas)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_accuracy_max = -np.Inf
        
    def __call__(self, val_acc, model):
        score = val_acc
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'Validation accuracy increased ({self.val_accuracy_max:.6f} --> {val_acc:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_accuracy_max = val_acc

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for images, meta, labels in pbar:
        images, meta, labels = images.to(device), meta.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, meta)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Optional: Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f'{running_loss/total:.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss/len(train_loader), correct/total

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, meta, labels in val_loader:
            images, meta, labels = images.to(device), meta.to(device), labels.to(device)
            
            outputs = model(images, meta)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss/len(val_loader), correct/total

def train_model_with_scheduler_and_checkpoint(
    model, train_loader, val_loader, optimizer, criterion, device, 
    epochs=20, patience=5, scheduler_patience=5, checkpoint_dir='checkpoints'):
    
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, 'mobilenetv3.pt')
    
    early_stopping = EarlyStopping(
        patience=patience, 
        verbose=True, 
        path=checkpoint_path
    )
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='max',  # Changed to max since we're monitoring accuracy
        patience=scheduler_patience, 
        verbose=True,
        factor=0.1,
        min_lr=1e-6
    )
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'lr': []
    }
    
    best_model_epoch = None
    
    for epoch in range(epochs):
        print(f'\nEpoch {epoch+1}/{epochs}')
        
        # Training phase
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        
        # Validation phase
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Update scheduler based on validation accuracy
        scheduler.step(val_acc)
        
        # Early stopping check
        early_stopping(val_acc, model)
        if val_acc > early_stopping.val_accuracy_max:
            best_model_epoch = epoch + 1
            
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    # Load best model
    model.load_state_dict(torch.load(checkpoint_path))
    
    # Plot training curves
    plot_training_curves_with_checkpoint(history, best_model_epoch)
    
    return model, history

def plot_training_curves_with_checkpoint(history, best_model_epoch):
    epochs_range = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
    
    # Loss curves
    ax1.plot(epochs_range, history['train_loss'], label='Training Loss')
    ax1.plot(epochs_range, history['val_loss'], label='Validation Loss')
    if best_model_epoch:
        ax1.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    
    # Accuracy curves
    ax2.plot(epochs_range, history['train_acc'], label='Training Accuracy')
    ax2.plot(epochs_range, history['val_acc'], label='Validation Accuracy')
    if best_model_epoch:
        ax2.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.legend()
    
    # Learning rate curve
    ax3.plot(epochs_range, history['lr'], label='Learning Rate')
    if best_model_epoch:
        ax3.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_title('Learning Rate Schedule')
    ax3.set_yscale('log')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Function to evaluate test metrics
def evaluate_test_metrics(model, test_loader, device):
    true_labels, pred_labels = test(model, test_loader, device)
    acc = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average='macro')
    recall = recall_score(true_labels, pred_labels, average='macro')
    f1 = f1_score(true_labels, pred_labels, average='macro')
    return acc, precision, recall, f1

# Placeholder for results
results = {
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1_score": []
}

# Run experiment for 3 random seeds
seeds = [42, 123, 569]  # Example random seeds
for seed in seeds:
    print(f"\nTraining with random seed: {seed}")
    set_random_seed(seed)
    
    # Reinitialize model, optimizer, and criterion
    model = EarlyFusionMobileNetV3(input_dim_meta, num_classes).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    
    # Train the model
    model, history = train_model_with_scheduler_and_checkpoint(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epochs=100,
        patience=7,
        scheduler_patience=3,
        checkpoint_dir='D:\\PAD-UFES\\checkpoints'
    )
    
    # Evaluate on test set
    acc, precision, recall, f1 = evaluate_test_metrics(model, test_loader, device)
    print(f"Seed {seed}: Accuracy={acc:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1 Score={f1:.4f}")
    
    # Save metrics
    results["accuracy"].append(acc)
    results["precision"].append(precision)
    results["recall"].append(recall)
    results["f1_score"].append(f1)

# Compute average and standard deviation
metrics_summary = {}
for metric, values in results.items():
    avg = np.mean(values)
    std_dev = np.std(values)
    metrics_summary[metric] = (avg, std_dev)
    print(f"{metric.capitalize()}: Mean={avg:.4f}, StdDev={std_dev:.4f}")

# Save the best model
best_seed_idx = np.argmax(results["accuracy"])
best_seed = seeds[best_seed_idx]
print(f"Best model was trained with seed {best_seed}, Accuracy={results['accuracy'][best_seed_idx]:.4f}")
torch.save(model.state_dict(), 'D:\\PAD-UFES\\best_early_fusion_mobilenetv3AdasynDA.pth')

In [ ]:
model = EarlyFusionMobileNetV3(input_dim_meta, num_classes).to(device)
model.load_state_dict(torch.load('D:\\PAD-UFES\\best_early_fusion_mobilenetv3smoteDA.pth'))
model.eval()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

params_m = count_parameters(model) / 1e6
print(f"Total Trainable Parameters: {params_m:.3f} M")


from ptflops import get_model_complexity_info
import torch.nn as nn

# Create a wrapper so ptflops sees a single input
class FusionWrapper(nn.Module):
    def __init__(self, model, meta_dim=59):
        super().__init__()
        self.model = model
        self.meta_dim = meta_dim

    def forward(self, x):
        batch = x.shape[0]
        dummy_meta = torch.randn(batch, self.meta_dim).to(x.device)
        return self.model(x, dummy_meta)

meta_dim = 59
wrapper = FusionWrapper(model, meta_dim).to(device)

with torch.no_grad():
    flops, params = get_model_complexity_info(
        wrapper,
        (3, 224, 224),
        as_strings=False,
        print_per_layer_stat=False,
        verbose=False
    )

print(f"FLOPs: {flops/1e9:.3f} GFLOPs")

import torch
import time
import numpy as np

def measure_gpu_latency(model, device, meta_dim=59, runs=200):
    model.eval()
    dummy_img = torch.randn(1, 3, 224, 224).to(device)
    dummy_meta = torch.randn(1, meta_dim).to(device)

    # Warm-up
    for _ in range(20):
        _ = model(dummy_img, dummy_meta)
    torch.cuda.synchronize()

    times = []
    for _ in range(runs):
        start = time.time()
        _ = model(dummy_img, dummy_meta)
        torch.cuda.synchronize()
        times.append((time.time() - start) * 1000)

    mean, std = np.mean(times), np.std(times)
    return mean, std, 1000 / mean

gpu_mean, gpu_std, gpu_fps = measure_gpu_latency(model, device, meta_dim=59)
print(f"GPU Latency: {gpu_mean:.3f} ± {gpu_std:.3f} ms")
print(f"GPU FPS: {gpu_fps:.2f}")


def measure_cpu_latency(model, meta_dim=59, runs=100):
    model_cpu = model.cpu()
    model_cpu.eval()

    dummy_img = torch.randn(1, 3, 224, 224)
    dummy_meta = torch.randn(1, meta_dim)

    # Warm-up
    for _ in range(10):
        _ = model_cpu(dummy_img, dummy_meta)

    times = []
    for _ in range(runs):
        start = time.time()
        _ = model_cpu(dummy_img, dummy_meta)
        times.append((time.time() - start) * 1000)

    mean, std = np.mean(times), np.std(times)
    return mean, std, 1000 / mean

cpu_mean, cpu_std, cpu_fps = measure_cpu_latency(model, meta_dim=59)
print(f"CPU Latency: {cpu_mean:.3f} ± {cpu_std:.3f} ms")
print(f"CPU FPS: {cpu_fps:.2f}")


In [ ]:
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def test(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            outputs = model(imgs, metas)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

true_labels, pred_labels = test(model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt=".0f", xticklabels = class_names, yticklabels = class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
fpr = dict()
tpr = dict()
roc_auc = dict()

n_class = 6

for i in range(n_class):
    fpr[i], tpr[i], _ = roc_curve(np.array(true_labels) == i, np.array(pred_labels) == i)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
colors = ['orange', 'green', 'red', 'blue', 'purple', 'pink']

for i in range(n_class):
    plt.plot(fpr[i], tpr[i], color=colors[i], lw=2, label=f'ROC curve (AUC = {roc_auc[i]:.2f}) for {class_names[i]}')

plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('(ROC)Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()